### Luke Waninger
Exploring a dataset containing call center tickets.

### Notebook init

In [1]:
%matplotlib inline
%config IPCompleter.greedy=True

import datetime as dt
import itertools
from IPython.core.interactiveshell import InteractiveShell
import multiprocessing
from multiprocessing import Lock, Manager, Process, Queue
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as figf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from scipy.stats import mode, norm
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import time
from tqdm import tqdm, tqdm_pandas

InteractiveShell.ast_node_interactivity = 'all'

proc_manager = Manager()
tqdm.pandas()
init_notebook_mode(connected=True)


UNKNOWN = 'unk'

## Notes:
* change the `cpu_count` variable below if you need to use your computer for other tasks while this notebook runs
* to view visualizations this notebook must be ran in Trusted mode
* Python version = 3.6

In [2]:
cpu_count = multiprocessing.cpu_count()
print(f'cores to utilize:   {cpu_count} of {multiprocessing.cpu_count()}')

cores to utilize:   12 of 12


## Load and clean the data

### Initial evaluation

In [4]:
df = pd.read_csv('tickets.csv')
df.head(5)

,Unnamed: 0,Issue_Resolution,Issue_Category,Issue_Severity,Support_Level,Support_Channel,Building,City,Position,Country,Tenure_Months,Career_Desc,Division
0,23935,Workaround,Apps,Sev1,Gold,Chat,MOBILE,Sydney,Inside Sales,AU,6.0,IC3,D009170
1,27259,User Education,Apps,Sev1,NaN,Chat,MOBILE,Edinburgh,Relationship Management,GB,52.0,IC4,D010405
2,52587,Resolved by Caller,Apps,Sev2,NaN,Chat,B0767,Zaventem,Sales Excellence,BE,3.0,NaN,D010467
3,51194,User Education,Apps,Sev3,Silver,Email,MOBILE,Melbourne,Relationship Management,AU,64.0,IC4,D009170
4,32585,Unknown,Apps,Sev3,Platinum,Chat,MOBILE,Sydney,Account Management,AU,34.0,IC2,D009170


At first glance, we see that most of the raw features are categorical or ordinal variables. In fact, the only quantitative variable is $\texttt{Tenure_Months}$. Another interesting characteristic of this data is the logical division of sources. On the left we have information regarding the ticket while the right contains information regarding either the agent on call or the individual who generated the ticket. Intrinsically, we have a one to many relationship between tickets and personnel. Separating these two datasets with correct associations gives more room for feature engineering.

In [5]:
df.columns = list(map(str.lower, df.columns))
col_map = {
    'issue_resolution': 'resolution',
    'issue_category':   'category',
    'issue_severity':   'severity',
    'support_level':    'level',
    'support_channel':  'channel',
    'tenure_months':    'tenure',
    'career_desc':      'career'
}
df.rename(columns=col_map, inplace=True)

tick_cols = [
    'unnamed: 0', 
    'resolution', 
    'category', 
    'severity', 
    'level',
    'channel',  
]

tech_cols = [
    'building', 
    'city', 
    'position', 
    'country', 
    'tenure', 
    'career', 
    'division'
]

numeric_cols = [
    'unnamed: 0',
    'tenure'
]

# strip whitespace and convert all cells to lowercase
for col in df.columns:
    if col in numeric_cols:
        continue
    
    else:
        df[col] = df[col].str.strip()
        df[col] = df[col].str.lower()

# separate tickets and technicians into separate dataframes
tickets = df.loc[:, tick_cols]
technicians = df.loc[:, tech_cols]

# check whether the unnamed ticket column is a valid key, rename if so
if (len(df.loc[:, 'unnamed: 0']) == len(pd.unique(df.loc[:, 'unnamed: 0']))):
    tmap = {'unnamed: 0':'ticket_id'}
    tickets.rename(columns=tmap, inplace=True)
    df.rename(columns=tmap, inplace=True)
    tick_cols[0] = 'ticket_id'
   
# drop duplicate technicians
technicians = technicians.drop_duplicates(keep='first')

tickets.head(5)
technicians.head(5)

,ticket_id,resolution,category,severity,level,channel
0,23935,workaround,apps,sev1,gold,chat
1,27259,user education,apps,sev1,NaN,chat
2,52587,resolved by caller,apps,sev2,NaN,chat
3,51194,user education,apps,sev3,silver,email
4,32585,unknown,apps,sev3,platinum,chat


,building,city,position,country,tenure,career,division
0,mobile,sydney,inside sales,au,6.0,ic3,d009170
1,mobile,edinburgh,relationship management,gb,52.0,ic4,d010405
2,b0767,zaventem,sales excellence,be,3.0,NaN,d010467
3,mobile,melbourne,relationship management,au,64.0,ic4,d009170
4,mobile,sydney,account management,au,34.0,ic2,d009170


The column `unnamed: 0` turned out to be valid key which further implies that no duplicate tickets existed in the frame. Before we do any further data reorganization we need to create a foreign key from ticket to technician in order to maintain data integrity.

In [6]:
tmp = dict()

techs = []
for idx, row in tqdm(
    zip(df.index.tolist(), df.iterrows()), 
    total=len(df), 
    desc='generating keys'
):
    key = '.'.join([str(a) for a in row[1][tech_cols]])
    
    if key not in tmp.keys():
        tmp[key] = idx
        val = idx
    else:
        val = tmp[key]
    
    techs.append(val)

tickets['technician'] = techs
tickets.head(5)
print(len(pd.unique(tickets.technician)))

del tmp, techs

generating keys: 100%|██████████| 10006/10006 [00:06<00:00, 1558.04it/s]


,ticket_id,resolution,category,severity,level,channel,technician
0,23935,workaround,apps,sev1,gold,chat,0
1,27259,user education,apps,sev1,NaN,chat,1
2,52587,resolved by caller,apps,sev2,NaN,chat,2
3,51194,user education,apps,sev3,silver,email,3
4,32585,unknown,apps,sev3,platinum,chat,4


8569


We've found around 2500 duplicate persons and mapped them as foreign keys.

In [7]:
pd.crosstab(tickets.resolution, tickets.category)

category,accounts,apps,help,infrastructure,other,phone,software
resolution,,,,,,,
hardware repair,0,9,0,0,0,0,0
known error,0,0,0,0,0,0,2
non-actionable,0,61,291,0,0,0,0
non-reproducible,0,83,2,1,0,0,0
reimaged,0,3,0,0,0,0,0
request complete,0,603,15,5,0,0,0
resolved by caller,0,463,12,2,0,2,0
software update,0,639,4,4,0,2,0
unknown,0,1764,15,10,0,4,0


### Support tickets

In [8]:
evaluation(tickets, exclusions=['ticket_id', 'technician'])

level
-----------------------------
NaN         8885
silver       929
gold         140
platinum      52
Name: level, dtype: int64

resolution
-----------------------------
workaround            3597
user education        2338
unknown               1811
software update        649
request complete       623
resolved by caller     504
non-actionable         352
non-reproducible        86
NaN                     26
hardware repair          9
user educate             4
reimaged                 3
user edacution           2
known error              2
Name: resolution, dtype: int64

category
-----------------------------
apps              9456
help               433
NaN                 53
infrastructure      38
phone               17
accounts             5
software             3
other                1
Name: category, dtype: int64

channel
-----------------------------
email    5305
chat     3812
phone     889
Name: channel, dtype: int64

severity
-----------------------------
sev2    6099
sev1

We immediately see several commonalities among the responses that we can take advantage of. Particularly, the signal regarding our research question can be increased by binning appropriate columns and reducing the number of features. Additionally, the 'severity' column has an embedded ordering so I'll change it to integer values and treat it as an ordinal variable.

In [9]:
# adjust the misspelling
edu_forms = ['user education', 'user edacution', 'user educate']
tickets['edu'] = tickets.resolution.isin(edu_forms).astype(int)

# drop resolution in favor of a binary 'is_edu'
tickets = tickets.drop(labels=['resolution'], axis=1)

# group the tiny categories
tickets.loc[tickets.category.isin(['accounts', 'software', 'other']) | tickets.category.isnull(), 'category'] = 'other'

# adjust the severity to make use of the implicit ordering and reset
# the single sev4 to sev3
tickets.loc[tickets.severity == 'sev1', 'severity'] = 1
tickets.loc[tickets.severity == 'sev2', 'severity'] = 2
tickets.loc[tickets.severity == 'sev3', 'severity'] = 3
tickets.loc[tickets.severity == 'sev4', 'severity'] = 3

# set the index to ticket_ids
tickets.index = tickets.ticket_id
tickets.drop(labels=['ticket_id'], axis=1, inplace=True)

In [10]:
print(f'within support level, {np.round(np.sum(tickets.level.isna()/len(tickets)), 3)*100}% are nan')

within support level, 88.8% are nan


The support_level column is almost exclusively NaN. Without background knowledge these values could come from two sources: one, the caller didn't have a support plan or two, the data was lost somewhere along the way. For this project, I'm assuming the first and setting the NaNs to 'none'.

In [11]:
tickets.loc[tickets.level.isnull(), 'level'] = 'none'

evaluation(tickets, exclusions=['technician'])

level
-----------------------------
none        8885
silver       929
gold         140
platinum      52
Name: level, dtype: int64

edu
-----------------------------
0    7662
1    2344
Name: edu, dtype: int64

category
-----------------------------
apps              9456
help               433
other               62
infrastructure      38
phone               17
Name: category, dtype: int64

channel
-----------------------------
email    5305
chat     3812
phone     889
Name: channel, dtype: int64

severity
-----------------------------
2    6099
1    2016
3    1891
Name: severity, dtype: int64

excluded NaN counts
-----------------------------
technician - 0


### Technicians

#### initial munging and imputation

In [12]:
evaluation(technicians, exclusions=[
    'city', 'country', 'division', 'tenure', 'position', 'building'
])

career
-----------------------------
ic4      3222
ic3      2388
ic5       991
ic2       798
mgr3      400
NaN       184
mgr2      156
mgr4      149
ic6       122
ic1       102
exec5      20
exec4      16
mgr5       13
mgr6        3
exec6       2
exec3       2
ic7         1
Name: career, dtype: int64

excluded NaN counts
-----------------------------
city - 141
country - 3
division - 0
tenure - 6
position - 0
building - 0


Fortunately, there aren't many missing values and of those, city and country might be linear combinations of other fields. First, impute the missing cities and countries by their inherent hierarchy. Find matching buildings and fall back to countries that must match.

In [13]:
missing_cities = technicians.loc[technicians.city.isnull(), :]
match_count = 0

# iterate through each row looking for matching cities by the building
# excluding the values 'home office' and 'mobile'
for idx, mc in missing_cities.iterrows():
    if mc.building == 'home office' or mc.building == 'mobile':
        technicians.loc[idx, 'city'] = UNKNOWN
        
    matches = technicians.loc[
        (~technicians.city.isnull()) &
        (technicians.country == mc.country)
    ]
    
    # group by the building, find, and set the city
    grouped = matches.groupby(by=['building'], axis=0)
    if mc.building in grouped.groups.keys():
        
        # get the group of technicians that work in the same building
        group = technicians.loc[grouped.groups[mc.building], :]
        
        # find and set the matching city
        city = pd.Series(group.city.value_counts()).values.argmax
        city = group.city.values[city()]
        
        technicians.loc[idx, 'city'] = city
        match_count += 1
    else:
        technicians.loc[idx, 'city'] = UNKNOWN

print(f'{match_count} match(es) found\n')
evaluation(technicians, exclusions=tech_cols)

137 match(es) found

excluded NaN counts
-----------------------------
building - 0
city - 0
position - 0
country - 3
tenure - 6
career - 184
division - 0


Next, impute missing countries by referencing technicians with matching cities.

In [14]:
missing_countries = technicians.loc[technicians.country.isnull(), :]
match_count = 0

# iterate through each row looking for matching cities
for idx, mc in missing_countries.iterrows():        
    matches = technicians.loc[
        (technicians.city != UNKNOWN) &
        (technicians.city == mc.city)
    ]
    
    # group by the city and find the country it's in
    grouped = matches.groupby(by=['city'], axis=0)
    if mc.city in grouped.groups.keys():
        # get the matching city, then set the matching country
        group = technicians.loc[grouped.groups[mc.city], :]
        country = pd.Series(group.country.value_counts()).values.argmax
        country = group.country.values[country()]
        technicians.loc[idx, 'country'] = country
        match_count += 1
    else:
        technicians.loc[idx, 'country'] = UNKNOWN
    
print(f'{match_count} match(es) found\n')

0 match(es) found



#### feature engineering

In [15]:
# create a ticket count
ticket_counts = pd.DataFrame(tickets.groupby(['technician']).aggregate(len)).iloc[:, 0]
technicians['ticket_count'] = np.ones(len(technicians))

for i, v in ticket_counts.iteritems():
    technicians.loc[i, 'ticket_count'] = v

# create binary variables for executive, managers, and ic
exec_ = ['exec6', 'exec5', 'exec4', 'exec3']
technicians['is_exec'] = [1 if x in exec_ else 0 for x in technicians.career.values]

mgr = ['mgr6', 'mgr5', 'mgr4']
technicians['is_mgr'] = [1 if x in mgr else 0 for x in technicians.career.values]

ic = ['ic7', 'ic6', 'ic5']
technicians['is_ic'] = [1 if x in ic else 0 for x in technicians.career.values]

t = [tech_cols.append(key) for key in ['ticket_count', 'is_exec', 'is_mgr', 'is_ic']]

#### binning
I join the underrepresented categories for two reasons: one, to give them more influence during training and two: to prevent errors during shuffles!

In [16]:
technicians.loc[technicians.career.isin(exec_), 'career'] = 'exec'
technicians.loc[technicians.career.isin(mgr), 'career'] = 'mgr4'
technicians.loc[technicians.career.isin(ic), 'career'] = 'ic5'

# group the building numbers to 'office' and rename 'home office' to 'home'
technicians.loc[technicians.building.str.startswith('b'), 'building'] = 'office'
technicians.loc[technicians.building == 'home office', 'building'] = 'home'

del exec_, mgr, ic

And finally, tenure has only 6 missing values from what should theoretically be an exponential distribution. We can impute these by finding means of correlated observations.

In [17]:
technicians.loc[technicians.tenure.isnull(), :]

,building,city,position,country,tenure,career,division,ticket_count,is_exec,is_mgr,is_ic
1183,mobile,san francisco,unassigned,us,NaN,NaN,d015742,1.0,0,0,0
2349,mobile,elk grove village,unassigned,us,NaN,NaN,d006433,2.0,0,0,0
2955,mobile,makati city,unassigned,ph,NaN,NaN,d008459,1.0,0,0,0
6827,office,unk,software engineering,unk,NaN,NaN,d010405,1.0,0,0,0
7020,office,unk,unassigned,unk,NaN,NaN,d013149,1.0,0,0,0
9135,mobile,shanghai,business evangelist,cn,NaN,ic4,d003414,1.0,0,0,0


In [18]:
# set the missing tenure values to the mean of associated data points
mean_tenure = np.mean(technicians.loc[
    ((technicians.country == 'ph') |
    (technicians.country == 'us')) & 
    ((technicians.building == 'mobile') |
    (technicians.building == 'b0535')), 'tenure']
)
technicians.loc[technicians.tenure.isnull(), 'tenure'] = mean_tenure

### Use LASSO to impute the missing careers

I use LASSO regression with this report because the features are almost exclusively categorical. Many indicator variables will be made and many of which will have no use in the final model. LASSO gives us a way to completely remove their influence.

##### prepare data

In [19]:
x  = technicians.loc[~technicians.career.isnull(), :].dropna()
xp = technicians.loc[technicians.career.isnull(), :].fillna(0)
y  = x.career.values

# generate a label->int mapping
labels = pd.unique(y)
label_map   = {i:a for a, i in enumerate(labels)}
label_map_r = {a:i for a, i in enumerate(labels)}
y = np.array([label_map[yi] for yi in y])

# drop the useless features
x  =  x.drop(labels=['career'], axis=1)
xp = xp.drop(labels=['career'], axis=1)

# generate indicator variables
x  = pd.get_dummies(x)
xp = pd.get_dummies(xp)

# add the missing sparse features
miss_xp = set(x.columns)-set(xp.columns)
miss_x  = set(xp.columns)-set(x.columns)
for col in miss_xp:
    xp[col] = np.zeros(len(xp))

for col in miss_x:
    x[col] = np.zeros(len(x))
        
# split and standardize
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=10)

scaler  = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test  = scaler.transform(x_test)
xp = scaler.transform(xp)

##### train

In [20]:
# generate a list of all the OvO pairs we need to train
pairs = [p for p in itertools.combinations(np.unique(y), 2)]

# setup a progress bar
pbar, qu = progress_bar(total=len(pairs), desc='fitting OvO classifiers')
                        
# setup the classifier
parameters = {
    'alpha': np.linspace(0.1, 5, 10),
    'fit_intercept': [True, False],
}

# fit using the producer consumer parallel pattern
clf = Lasso(max_iter=5000, selection='random', copy_X=True)
clfs = prod_con_map(
    func=fit_one, 
    vals=[(pair, qu, clf, parameters) for pair in pairs], 
    n_cons=cpu_count
)

# terminate and close the progres bar
pbar.terminate()
pbar.join()

fitting OvO classifiers: 100%|██████████| 36/36 [06:32<00:00,  6.07s/it]

##### validate

In [21]:
# make predictions
def predict_ovo(x):
    yhat, rcoefs = [], []
    for clf in clfs:
        clf, pos, neg = clf
        plab, nlab = label_map_r[pos], label_map_r[neg]

        # show the remaining number of coefficients so we can get a 
        # feeling for the effectiveness of our feature space
        rc = np.array([i for i, c in enumerate(clf.best_estimator_.coef_) if c > 0])
        rcoefs.append(rc)

        t = clf.predict(x)
        yhat.append([pos if yhi > 0 else neg for yhi in t])

    # take the mode of predicted values, breaking ties at random-uniform
    yhat = np.array(yhat).T
    yhat = [np.random.choice(mode(r).mode, 1)[0] for r in yhat]    

    # smoosh the remaining coefficients
    coefs = []
    for l in rcoefs:
        for e in l:
            coefs.append(e)
    rcoefs = coefs
    
    return yhat, rcoefs

yhat, rcoefs = predict_ovo(x_test)

# show the confusion matrix
cm = confusion_matrix(y_test, yhat)

fig = plot_confusion_matrix(cm, classes=labels)
iplot(fig, filename='cm')

# print validation score
print(f'\nfinal validation error: {np.mean(yhat != y_test)}')
print(f'of {x_train.shape[1]} features, only {len(np.unique(rcoefs))} were found useful for any model')


final validation error: 0.43156890796375774
of 800 features, only 20 were found useful for any model


##### impute
The validation error isn't to bad considering the number of classes we're predicting. Also note that IC4 tends to pull in the IC3 labels which could indicate that we're underfitting for IC4 or overfitting IC3. I tried several down/up sampling techniques to adjust for this but in the decided that a different model might be a better choice. Let's go ahead and impute the remaining null career entries using the trained models.

In [22]:
yhat, tmp = predict_ovo(xp)

# set the imputed
idx = technicians.career.isnull()
technicians.loc[idx, 'career'] = [label_map_r[yhi] for yhi in yhat]

del yhat, tmp

### Final data set

In [23]:
evaluation(tickets, exclusions=['technician'])

level
-----------------------------
none        8885
silver       929
gold         140
platinum      52
Name: level, dtype: int64

edu
-----------------------------
0    7662
1    2344
Name: edu, dtype: int64

category
-----------------------------
apps              9456
help               433
other               62
infrastructure      38
phone               17
Name: category, dtype: int64

channel
-----------------------------
email    5305
chat     3812
phone     889
Name: channel, dtype: int64

severity
-----------------------------
2    6099
1    2016
3    1891
Name: severity, dtype: int64

excluded NaN counts
-----------------------------
technician - 0


In [24]:
evaluation(technicians, exclusions=['city', 'country', 'division', 'tenure', 'position'])

career
-----------------------------
ic4     3309
ic3     2485
ic5     1114
ic2      798
mgr3     400
mgr4     165
mgr2     156
ic1      102
exec      40
Name: career, dtype: int64

is_exec
-----------------------------
0    8529
1      40
Name: is_exec, dtype: int64

building
-----------------------------
office    4706
mobile    3298
home       565
Name: building, dtype: int64

is_ic
-----------------------------
0    7455
1    1114
Name: is_ic, dtype: int64

ticket_count
-----------------------------
1.0     7713
2.0      603
3.0      132
4.0       49
5.0       28
6.0       15
7.0        7
8.0        7
10.0       5
11.0       3
9.0        3
12.0       3
14.0       1
Name: ticket_count, dtype: int64

is_mgr
-----------------------------
0    8404
1     165
Name: is_mgr, dtype: int64

excluded NaN counts
-----------------------------
city - 0
country - 0
division - 0
tenure - 0
position - 0


## Visual Analysis

First I'd like to make a visual assessment of each indicator. Is it clear that any variable behaves differently for education vs. non-education resolution types?

In [25]:
# plot predictors w/respect to response
df = tickets.join(technicians, on='technician', how='outer')
predictors = [
    'channel', 'severity', 'category', 'level', 'is_exec', 'is_mgr', 'is_ic', 'building'
]

# setup the main figure
fig = tools.make_subplots(
    rows=3, cols=5,
    specs=[
        [{'rowspan':3}, {'colspan':4}, None, None, None],
        [None, {}, {}, {}, {}],
        [None, {}, {}, {}, {}]
    ],
    subplot_titles=(
            'Resolved through education',
            'Proportions of predictors resolved through education'
    ),
    print_grid=False
)

# update subplot sizes
fig['layout']['yaxis2'].update(domain=[.95, 1])
t = [fig['layout'][f'yaxis{i}'].update(domain=[.55, .95]) for i in [3, 4, 5,  6]]
t = [fig['layout'][f'yaxis{i}'].update(domain=[0,  .35])  for i in [7, 8, 9, 10]]

# remove the extra ink
for idx in [4, 5, 6, 8, 9, 10]:
    fig['layout'][f'yaxis{idx}'].update(
        ticks='',
        showline=False,
        zeroline=False,
        showticklabels=False
    )

# generate the main leftside bars and add them to the plot
mb0 = go.Bar(
        name='other resolution types',
        x=['no'],
        y=tickets.loc[tickets.edu == 0, 'edu'].value_counts(),
        marker=dict(
            color=Theme().GREY
        ),
        xaxis='x1',
        yaxis='y1'
    )
mb1 = go.Bar(
        name='solved through user education',
        x=['yes'],
        y=tickets.loc[tickets.edu == 1, 'edu'].value_counts(),
        marker=dict(
            color=Theme().ORANGE
        ),
        xaxis='x1',
        yaxis='y1'
    )

t = fig.append_trace(mb0, 1, 1)
t = fig.append_trace(mb1, 1, 1)

# create the indicator bars
col_idx = 2
for ax, predictor in enumerate(predictors, 3):
    row = 2 if 3 <= ax <= 6 else 3
    col = ax-1 if 3 <= ax <= 6 else ax-5
    bars = [] 
    
    for edu in [0, 1]:
        series = df.loc[df.edu == edu, predictor]
        indicators = pd.unique(df[predictor])    

        for i, indicator in enumerate(indicators): 
            a = len(series[series == indicator])
            b = len(series)
            y = a/(a+b)

            b = go.Bar(
                name=indicator,
                x=['no' if edu == 0 else 'yes'],
                y=[y],
                text=indicator,
                marker=dict(color=Theme()[i]),
                xaxis=f'x{ax}',
                yaxis=f'y{ax}'
            )
            
            fig.append_trace(b, row, col)
    fig['layout'][f'xaxis{ax}'].update(title=predictor)
    
fig['layout'].update(barmode='stack', showlegend=False)
iplot(fig, filename='issue_bar')

As you can see, theres almost no discernible difference between the predictors when separated by response variable. Since we have geolocation data, lets get an idea of where the positive labels are coming from by generating a cross-tabulation/bar plot and choropleth.

In [26]:
# mapping country codes to names
cc = pd.read_csv('country_codes.csv')
cc.columns = ['code', 'name']
code_map = {str(code).lower():name for code, name in zip(cc.code.values, cc.name)}

df['country'] = [code_map[v] if v in code_map.keys() else v for v in df.country.values]

# create the cross-tabulation, normalized
ct = pd.crosstab(df.country, df.edu, normalize='index')
ct.drop(columns=0, inplace=True)

# create a non-normalized cross tab to get the count of tickets per country
temp = pd.crosstab(df.country, df.edu)
ct['tickets'] = [a+b for a, b in zip(temp.iloc[:, 0], temp.iloc[:, 1])]
ct = ct.sort_values(by=1, axis=0, ascending=False)

# calculate the entire sample proportion
total_prop = np.round(len(df.loc[df['edu'] == 1, :])/len(df.loc[df['edu'] == 0, :]), 3)
print(f'The proportion of all tickets closed through education is {total_prop}.')

n_countries = 25
x = ct.index.values[2:n_countries]
y = ct.iloc[2:n_countries, 0]

data = go.Bar(
        x=x,
        y=y,
        marker=dict(
            color=Theme().GREY
        ),
        xaxis='x1',
        yaxis='y1'
    )
layout = go.Layout(
        title = 'Top performing countries',
        xaxis = dict(tickangle = 45),
        yaxis = dict(title='Proportion closed through education')
    )

fig = go.Figure(data=[data], layout=layout)
iplot(fig, filename='top_countries')
del temp

The proportion of all tickets closed through education is 0.306.


In [27]:
# plot by percentage of tickets closed through education
data = [
    go.Choropleth(
        locationmode = 'country names',
        locations = ct.index.values,
        z = ct.iloc[:, 0],
        colorscale = [[0, Theme().TAN],[1, Theme().GREY]],
        autocolorscale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            )),
        geo = 'geo'
      )
    ]
# generate the layout
layout = go.Layout(
    title = 'Percentage of tickets closed through education',
    geo = dict(
        scope = 'world',
        showcountries = True,
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'Mercator'
        )
    ))

fig = go.Figure(data=data, layout=layout )
iplot(fig, filename='world_map' )

In [28]:
# plot by number of tickets
data = [
    go.Choropleth(
        locationmode = 'country names',
        locations = ct.index.values,
        z = ct.tickets,
        colorscale = [[0, Theme().TAN],[1, Theme().GREY]],
        autocolorscale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            )),
        geo = 'geo'
      )
    ]

# generate the layout
layout = go.Layout(
    title = 'Number of tickets closed',
    geo = dict(
        scope = 'world',
        showcountries = True,
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'Mercator'
        )
    ))

fig = go.Figure(data=data, layout=layout )
iplot(fig, filename='world_map' )

The most notable countries are displayed above. The particularly interesting countries are those that also have a considerable amount of tickets. It could be useful for the leadership team to evaluate the difference in business processes between these and countries in the lower proportions.

Additionally, the United states has the highest number of tickets and is below the mean. Lets perform a large-value z-test to show how much statistical difference exists.

$H_0: \quad \hat{p}_{us} - \hat{p}_{world} = 0$

$H_a: \quad \hat{p}_{us} - \hat{p}_{world} \ne 0$

$Z: \frac{\hat{p}_{us}-\hat{p}_{world}}{\sqrt{\hat{p}(1-\hat{p}(1/n_{us}+1/n_{world}))}}$

In [29]:
# run the test
us = 'United States of America'
pa = ct.loc[us, 1]
na = len(df.loc[df.country == us, 'country'])
print(f'US proportion: {np.round(pa, 3)}, n: {na}')

pb = np.mean(ct.loc[~(ct.index == 'us'), 1])
nb = len(df.loc[df.country != us, 'country'])
print(f'world proportion: {np.round(pb, 3)}, n: {nb}\n')

n  = na + nb
ph = (n*pa+nb*pb)/(na+nb)
se = np.sqrt(ph*(1-ph)*(1/na+1/nb))
z  = abs(pa-pb)/se
p  = 2*norm.sf(abs(z))
dif   = pa-pb
lower = dif-1.96*se
upper = dif+1.96*se

print(f'standard error: {np.round(se, 4)}')
print(f'z score: {np.round(z, 4)}')
print(f'p value: {np.round(p, 4)}')
print(f'95% CI ({np.round(lower, 4)}, {np.round(upper, 5)})')

US proportion: 0.204, n: 4841
world proportion: 0.289, n: 5165

standard error: 0.0096
z score: 8.9045
p value: 0.0
95% CI (-0.1039, -0.06639)


The evidence strongly suggests that the proportion of tickets closed through education in the US is lower than the world mean proportion.

For our last predictor, tenure, lets create a scatter plot of the mean proportion of successful tickets.

In [30]:
# prop by tenure
x = np.unique(sorted(df.tenure.tolist()))
y = []
c = []

grouped = df.groupby('tenure', axis=0)
for key in grouped.groups.keys():
    a = np.sum(grouped.get_group(key).edu)
    b = len(grouped.get_group(key).edu)
    y.append(a/(a+b))
    c.append(a+b)

scat = go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    marker=dict(
        color = c,
        colorscale = [[0, Theme().GREY], [1, Theme().ORANGE]],
        showscale=True,
        opacity=.8
    )
)

layout = go.Layout(
    title='Proportion of positive response variable increases with tenure',
    xaxis=dict(title='Months of tenure'),
    yaxis=dict(title='Proportion of tickets closed through education')
)

fig = go.Figure(data=[scat], layout=layout)
iplot(fig, filename='tenure_props')

In [31]:
# density
mgr = df.loc[df.is_mgr  == True, 'tenure']
ic  = df.loc[df.is_ic   == True, 'tenure']
hist = [mgr, ic]

labels = ['manager', 'IC']
colors = [Theme().GREY, Theme().ORANGE]

fig = figf.create_distplot(hist, labels, bin_size=[5, 5], colors=colors)
fig['layout'].update(title='Density of tenure by career')
iplot(fig, filename='tenure_density')

We can draw some interesting conclusions from these visualizations. First, it appears as if the proportion of successful tickets begin to increase near 200 months of tenure. But more notably, most of the personnel in the range of increase are managers. Also notice that the majority of tickets being resolved are in the lower end of tenure meaning less mature, and lower ranking personnel are handling most tickets. This is somewhat expected behavior as managers rarely sit as on-call tech support.

Another interesting characteristic is that neither IC or manager densities resemble the distribution of what should be a Poisson process.

## Modeling

I'll generate a logistic regression model so we can see how each factor effects the probability of a ticket being closed through education. First, lets create the indicator variables and fit a LogisticRegression with L1 regularization. I'd like to see which variables contribute the most variance and eliminate those that do not.

In [32]:
# prepare the data
x = df.drop(columns=['edu', 'city', 'country', 'position', 'division', 'technician'])
y = df.edu.values

# generate indicator variables
x  = pd.get_dummies(x)
        
# split and standardize
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

scaler  = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test  = scaler.transform(x_test)

First, lets fit the model without regularization so we can interpret the coefficients. Scikit doesn't provide an option to not regularize so we'll set the regularization parameter to a level such that it'll make no difference.

In [33]:
model = LogisticRegression(
    penalty='l2', 
    C=1e9, 
    random_state=42
).fit(x_train, y_train)

In [34]:
# plot feature importance
coef = [dict(feature=b, coef=(a)) for a, b in zip(model.coef_[0], x.columns)]
coef = sorted(coef, key=lambda x: x['coef'])

bar = go.Bar(
    x = [x['feature'] for x in coef],
    y = [x['coef'] for x in coef],
    marker = dict(color = Theme().GREY),
    opacity = .9
)
layout = go.Layout(
    title='Feature importance',
    xaxis=dict(
        title='feature',
        tickangle=45
    ),
    yaxis=dict(title='magnitude of coefficient')
)
fig = go.Figure(data=[bar], layout=layout)
iplot(fig, filename='feature_importance')

This plot makes me want to engineer some more features. Notice the most impacting feature is 'ticket_count', an engineered. This says that, when all other covariates are held constant, the probability of a ticket solved through education diminishes by over 30% for each new ticket! The next most important feature that reduces our probability is an indicator determining whether or not the person worked in an office. On the other end of the spectrum we have a few features that give us a boost: category_apps, building_mobile, and career_ic2 are the top three at .1, .08, and .07 respectively.